In [35]:
B1 = load('blocos/bloco_1_linear_cubo'); ##Isso na formula  6.28 secao 6.4.1
B1 = B1.apply_map(lambda x: x.simplify_real().combine())
load("../util.sage")

##### 5. There exits a $\delta<1$ such $(O_1)_{12}(c(t),t)$ is negative for $t$ in $(0,\delta)$ and is positive for $t$ in $(\delta,1)$ .

In [36]:
p1_t = B1.det()

In [37]:
α0 = p1_t.coefficients()[0][0]
α1 = p1_t.coefficients()[1][0]

In [38]:
α0_new  = α0.simplify_real().combine()

In [39]:
α0_numerator = α0_new.numerator()

In [40]:
C12 = B1[0,1].subs(c = -α0/α1)
C12_diff = diff(C12, t)

In [41]:
assume(t>0, t<1)
C12_new_diff = C12_diff.numerator().simplify_real().expand().combine() # não zera em t=0 nem t=1

In [42]:
var('u,v')
assume(u>0)
assume(v>0)

In [43]:
#show(C12_new_diff )

In [44]:
C12_diff_uvt = C12_new_diff.subs({(3*t^2 - 2*t + 3):v^2}).subs({(3*t^2 + 2*t + 3):u^2}).simplify_real().expand().combine()

In [45]:
#Resolve 3*t^2 + 2*t + 3=u^2 em relação a u
sol_u = solve(u**2 == 3*t^2 + 2*t + 3, u)[0].rhs() 

#Resolve 3*t^2 - 2*t + 3=v^2 em relação a v
sol_v = solve(v**2 == 3*t^2 - 2*t + 3, v)[0].rhs()

# obtem os valores de u para t=0 e t=1
u0 = sol_u(t=0)
u1 = sol_u(t=1)

# obtem os valores de v para t=1/3 e t=1
v0 = sol_v(t=1/3)
v1 = sol_v(t=1)

u0, u1, v0, v1

(sqrt(3), 2*sqrt(2), 2*sqrt(2/3), 2)

In [46]:
poly_u_v_ico_num = C12_diff_uvt
graus_u_v_t  =  [poly_u_v_ico_num.degree(u), poly_u_v_ico_num.degree(v), poly_u_v_ico_num.degree(t)]
graus_u_v_t

[11, 11, 59]

In [50]:
from functools import reduce

def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario

def coeficientes_de_moebius_simbolico_varias_variaveis(poly, graus, extremos, novo_coefsxy ):
    """INPUTS -
    poly- Um polinomio como expressão simbolica na classe  <class 'sage.symbolic.expression.Expression'>;
    graus - Uma lista com naturais representando o multigrau do monomio que voce quer o coeficiente.
            Importante - A sua lista de graus deve estar na ordem que o Sage ordena as variaveis.
    extremos - Uma lista com pares (a_i,b_i) que dão o intervalo da substituição de Moebius para
    a variavel x_i, isto é,    x_i---> x_i = (a_i*u_i+b_i)/(u_i+1) para a variavel u_i.
    OUTPUT - O coeficiente do monomio com multigrau dado pela lista graus depois da substituicao de Moebius
    Exemplo - 1
    sage: poly_teste = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4;
    sage: graus_teste = [1,2]; ## para acahar o coeficiente de (x_1)^1*(x_2)^2
    sage: extremos_teste = [(3, 5), (7, 11)];
    sage: novo_coefsxy_teste = dicionario_geral(poly_teste); # virar variável global;
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly_teste, graus_teste, extremos_teste, novo_coefsxy_teste)
        1133944
    Exemplo - 2
    sage: poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = poly.free_variables(); variaveis
    sage: extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
    sage: ru, rx, ry, rz = var('ru, rx, ry, rz')
    sage: variaveis_r = [ru, rx, ry, rz]
    sage: novo_coefsxy_teste_2 = dicionario_geral(poly); 
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste_2) ## Procurando o coeficiente de ru^0 * rx^0 * ry^1 * rz^0
    sage: show(c)
     244374065874
       
       OBSERVACAO: Para mais detalhes veja o arquivo /Artigo_de_Aplicacoes/Reuniao-casa-marcelo-26-07/Icosaedro-bloco-2-reunião-casa-marcelo/Funcao_Para_Coeficientes_Com_Varias_Variaveis_Versao_Final.ipynb
        """
    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
    graus_total = [poly.degree(x) for x in variaveis];
    chaves = novo_coefsxy.keys()
    soma_total = 0
    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            somak = 0
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]
            for pp in range(ck+1):
                somak = somak+((binomial(ck,pp))*(binomial(gt-ck, gks-pp))*((a_k)^(pp))*((b_k)^(ck-pp)))
            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total = soma_total + reduce(lambda a, b: a*b, somas_e_valor)
    return soma_total

from functools import reduce
from sage.all import binomial

def dicionario_geral_gpt(poly):
    def nova_lista(lista, var):
        new_lista = []
        for item in lista:
            sublista = item[0].coefficients(var)
            for coef, monom in sublista:
                new_lista.append([coef, monom + item[-1]])
        return new_lista

    variaveis = poly.variables()
    variavel_1 = variaveis[0]
    l_variavel_1 = poly.coefficients(variavel_1)
    l = [[item[0], item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(reversed(j[1])): j[0] for j in l}
    return dicionario

def coeficientes_de_moebius_simbolico_varias_variaveis_gpt(poly, graus, extremos, novo_coefsxy):
    variaveis = poly.variables()
    graus_total = [poly.degree(x) for x in variaveis]
    chaves = novo_coefsxy.keys()
    soma_total = 0

    binomials_cache = {}
    powers_cache = {}

    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]

            somak = 0
            for pp in range(ck + 1):
                binom_ck_pp = binomials_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binomials_cache[(ck, pp)] = binom_ck_pp

                binom_gtck_gkspp = binomials_cache.get((gt - ck, gks - pp))
                if binom_gtck_gkspp is None:
                    binom_gtck_gkspp = binomial(gt - ck, gks - pp)
                    binomials_cache[(gt - ck, gks - pp)] = binom_gtck_gkspp

                a_k_pp = powers_cache.get((a_k, pp))
                if a_k_pp is None:
                    a_k_pp = a_k^pp
                    powers_cache[(a_k, pp)] = a_k_pp

                b_k_ck_pp = powers_cache.get((b_k, ck - pp))
                if b_k_ck_pp is None:
                    b_k_ck_pp = b_k^(ck - pp)
                    powers_cache[(b_k, ck - pp)] = b_k_ck_pp

                somak += binom_ck_pp * binom_gtck_gkspp * a_k_pp * b_k_ck_pp

            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total += reduce(lambda a, b: a * b, somas_e_valor)

    return soma_total





In [51]:
%%cython
from libc.math cimport pow
from functools import reduce
from sage.all import binomial

def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario
from functools import reduce

def coeff_de_moebius(poly, graus, graus_total, chaves, extremos, novo_coefsxy):    
    soma_total = 0

    binomials_cache = {}
    powers_cache = {}

    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]

            somak = 0
            for pp in range(ck + 1):
                binom_ck_pp = binomials_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binomials_cache[(ck, pp)] = binom_ck_pp

                binom_gtck_gkspp = binomials_cache.get((gt - ck, gks - pp))
                if binom_gtck_gkspp is None:
                    binom_gtck_gkspp = binomial(gt - ck, gks - pp)
                    binomials_cache[(gt - ck, gks - pp)] = binom_gtck_gkspp

                a_k_pp = powers_cache.get((a_k, pp))
                if a_k_pp is None:
                    a_k_pp = pow(a_k, pp)
                    powers_cache[(a_k, pp)] = a_k_pp

                b_k_ck_pp = powers_cache.get((b_k, ck - pp))
                if b_k_ck_pp is None:
                    b_k_ck_pp = pow(b_k,(ck - pp))
                    powers_cache[(b_k, ck - pp)] = b_k_ck_pp

                somak += binom_ck_pp * binom_gtck_gkspp * a_k_pp * b_k_ck_pp

            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total += reduce(lambda a, b: a * b, somas_e_valor)

    return soma_total


In [53]:
dicionario = dicionario_geral(poly_u_v_ico_num)

In [30]:
#dicionario_em_AA = {tuple([ZZ(k) for k in key]) : AA(value) for key, value in  dicionario.items()}


In [54]:
n_uplas = [[i,j,k] for i in range(graus_u_v_t[0]+1) for j in range(graus_u_v_t[1]+1) for k in range(graus_u_v_t[2]+1)]; len(n_uplas)

8640

In [55]:
poly = poly_u_v_ico_num
extremos = [(u0, u1), (v0, v1), (0, 1)]
variaveis = poly.variables()
graus_total = [poly.degree(x) for x in variaveis]
chaves = dicionario.keys()
novo_coefsxy = dicionario

In [ ]:
coeff = []
for i, grau in enumerate(n_uplas):
    
    c = coeff_de_moebius(poly, grau, graus_total, chaves, extremos, novo_coefsxy)
    coeff.append(c)
    if i%10==0:
        print(i)


#%time coeff = [coeff_de_moebius(poly, grau, graus_total, chaves, extremos, novo_coefsxy) for grau in n_uplas]

0
10
20
30
40
50


In [ ]:
coeficientes_em_AA

In [ ]:
dicionario_em_AA
